# Text Classification

This notebook is for specialized text classification, including some using deep learning via TorchSharp.

This is kept in its own notebook because it uses CUDA 12 and Linux

## Dependencies

In [1]:
#r "nuget:Microsoft.Data.Analysis"
#r "nuget:Microsoft.ML"
#r "nuget:Microsoft.ML.AutoML"
#r "nuget:Microsoft.ML.TorchSharp"
#r "nuget:Newtonsoft.Json"
#r "nuget:Plotly.NET"
#r "nuget:Plotly.NET.Interactive"
#r "nuget:libtorch-cpu-linux-x64, 2.1.0.1"
#r "nuget:TorchSharp,0.101.5"

using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.Data.Analysis;
using Microsoft.ML;
using Microsoft.ML.AutoML;
using Microsoft.ML.AutoML.CodeGen;
using Microsoft.ML.SearchSpace;
using Microsoft.ML.SearchSpace.Option;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using Microsoft.ML.Transforms;
using Microsoft.ML.Transforms.Text;
using Microsoft.ML.TorchSharp;
using Microsoft.ML.TorchSharp.NasBert;
using TorchSharp;
using Newtonsoft.Json;
using System.Reflection;

Installed Packages libtorch-cpu-linux-x64, 2.1.0.1 Microsoft.Data.Analysis, 0.21.1 Microsoft.ML, 3.0.1 Microsoft.ML.AutoML, 0.21.1 Microsoft.ML.TorchSharp, 0.21.1 Newtonsoft.Json, 13.0.3 Plotly.NET, 5.0.0 Plotly.NET.Interactive, 5.0.0 TorchSharp, 0.101.5

Loading extensions from `/home/matteland/.nuget/packages/plotly.net.interactive/5.0.0/lib/netstandard2.1/Plotly.NET.Interactive.dll`

Loading extensions from `/home/matteland/.nuget/packages/microsoft.data.analysis/0.21.1/interactive-extensions/dotnet/Microsoft.Data.Analysis.Interactive.dll`

Loading extensions from `/home/matteland/.nuget/packages/microsoft.ml.automl/0.21.1/interactive-extensions/dotnet/Microsoft.ML.AutoML.Interactive.dll`

Loading extensions from `/home/matteland/.nuget/packages/skiasharp/2.88.6/interactive-extensions/dotnet/SkiaSharp.DotNet.Interactive.dll`

In [3]:
//#r "nuget:MattEland.ML"
//#r "nuget:MattEland.ML.Charts"
//#r "nuget:MattEland.ML.DataFrames"
//#r "nuget:MattEland.ML.Interactive"
#r "/home/matteland/Documents/MattEland.ML/MattEland.ML/MattEland.ML/bin/Debug/net8.0/MattEland.ML.dll"
#r "/home/matteland/Documents/MattEland.ML/MattEland.ML/MattEland.ML.DataFrames/bin/Debug/net8.0/MattEland.ML.DataFrames.dll"
#r "/home/matteland/Documents/MattEland.ML/MattEland.ML/MattEland.ML.Charts/bin/Debug/net8.0/MattEland.ML.Charts.dll"
#r "/home/matteland/Documents/MattEland.ML/MattEland.ML/MattEland.ML.Interactive/bin/Debug/net8.0/MattEland.ML.Interactive.dll"

using MattEland.ML;
using MattEland.ML.Charts;
using MattEland.ML.DataFrames;
using MattEland.ML.Interactive;

await MattEland.ML.Interactive.InteractiveExtensions.Load(Microsoft.DotNet.Interactive.KernelInvocationContext.Current.HandlingKernel.RootKernel);

reflect 
 This extension generates a list of properties on the object in tabular format.

transformer-vis 
 This extension generates Flowcharts from ITransformers using the Mermaid kernel.

pipeline-vis 
 This extension generates Flowcharts from Pipelines using the Mermaid kernel.

## Data Loading / Splitting

In [4]:
DataFrame df = DataFrame.LoadCsv("data/Training.csv", separator: ',', header: true);
df.Columns.Remove("PredictedLabel", "Reasoning", "AuthorId", "AuthorDateUtc", "CommitterId", "CommitterDateUtc", "ParentSha", "Parent2Sha", "DayOfWeek", "Month", "Quarter", "Year", "Hour", "TimeOfDay", "IsWeekend", "Sha", "Source");
df["ActualLabel"].SetName("Label");
df.Sample(5)

index,Label,Message,IsMerge,WorkItems,TotalFiles,ModifiedFiles,AddedFiles,DeletedFiles,TotalLines,NetLines,AddedLines,DeletedLines,HasAddedFiles,HasDeletedFiles,MessageLength,WordCount
0,True,revert dependency on sdk acquisition and cache location of dotnet tool (#1094),False,1,6,6,0,0,1703,42,64,22,False,False,78,12
1,False,update system drawing,False,0,1,1,0,0,47,0,1,1,False,False,21,3
2,False,compute cursor start for replacement,False,0,1,1,0,0,238,1,2,1,False,False,36,5
3,False,Updated branch version (#5400),False,1,1,1,0,0,40,0,2,2,False,False,30,4
4,False,Merged PR 1684342: support for utf8 encoding and emoji,False,0,1,1,0,0,1,0,1,1,False,False,54,9


In [5]:
int seed = 42;
MLContext context = new(seed: seed);

var split = context.Data.TrainTestSplit(df, testFraction: 0.3, seed: seed);
var colTypes = df.GetColumnTypes(excludedColumns: new[] { "Label" });
colTypes

Text,[ Message ]
Numeric,"[ WorkItems, TotalFiles, ModifiedFiles, AddedFiles, DeletedFiles, TotalLines, NetLines, AddedLines, DeletedLines, MessageLength, WordCount ]"
Categorical,"[ IsMerge, HasAddedFiles, HasDeletedFiles ]"
Excluded,[ Label ]


## Simple Featurizer

In [11]:
MLContext context = new(seed: seed) {
    GpuDeviceId = 0,
    FallbackToCpu = true,
};
var featurizer = context.Auto().Featurizer(df);

// The classifier step tells AutoML what model trainers are enabled. We'll focus on those that don't require scaled data for simplicity at the moment
var classifier = context.Auto().BinaryClassification(
    useFastForest: true, 
    useLgbm: true, 
    useFastTree: true, 
    useLbfgsLogisticRegression: false, 
    useSdcaLogisticRegression: false);

// Now let's run our experiment using our custom pipeline
var experiment = context.Auto().CreateExperiment()
    .SetPipeline(featurizer.Append(classifier))
    .SetDataset(split)
    .SetBinaryClassificationMetric(BinaryClassificationMetric.F1Score, labelColumn: "Label")
    .SetMaxModelToExplore(10);

TrialResult result = await experiment.RunAsync();
ITransformer model = result.Model;

var scorer = model.Transform(split.TestSet);

// If the model supports calibration, we could use Evaluate instead
var evalResults = context.BinaryClassification.EvaluateNonCalibrated(model.Transform(split.TestSet), labelColumnName: "Label");

// Let's see how it performed
MLCharts.ClassificationReport(evalResults)

<!-- Plotly chart will be drawn inside this DIV -->

In [12]:
model

[ Microsoft.ML.Transforms.MissingValueReplacingTransformer, Microsoft.ML.Transforms.TypeConvertingTransformer, Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer, Microsoft.ML.Data.ColumnConcatenatingTransformer, Microsoft.ML.Data.BinaryPredictionTransformer`1[Microsoft.ML.Trainers.FastTree.FastForestBinaryModelParameters] ] LastTransformer Microsoft.ML.Data.BinaryPredictionTransformer`1[Microsoft.ML.Trainers.FastTree.FastForestBinaryModelParameters] FeatureColumnName Features FeatureColumnType Vector<Single, 5861> Dimensions [ 5861 ] IsKnownSize True ItemType Single RawType System.Single Size 5861 RawType Microsoft.ML.Data.VBuffer<System.Single> Model Microsoft.ML.Trainers.FastTree.FastForestBinaryModelParameters TrainedTreeEnsemble Microsoft.ML.Trainers.FastTree.QuantileRegressionTreeEnsemble Bias 0 TreeWeights [ 1, 1, 1, 1 ] Trees index value 0 Microsoft.ML.Trainers.FastTree.QuantileRegressionTree LeftChild [ 1, 2, -1 ] RightChild [ -2, -3, -4 ] NumericalSplitFeatureIndexes [ 1353, 103, 5857 ] NumericalSplitThresholds [ 0.057353932, 0.10190575, 9.5 ] CategoricalSplitFlags [ False, False, False ] LeafValues [ -0.5764705882352941, 0.9047619047619048, -1, 0.3333333333333333 ] SplitGains [ 43.45517488043252, 8.790652661395029, 11.409432962374133 ] NumberOfLeaves 4 NumberOfNodes 3 1 Microsoft.ML.Trainers.FastTree.QuantileRegressionTree LeftChild [ 1, 2, -1 ] RightChild [ -2, -3, -4 ] NumericalSplitFeatureIndexes [ 1353, 106, 5850 ] NumericalSplitThresholds [ 0.057353932, 0.1075328, 0.5 ] CategoricalSplitFlags [ False, False, False ] LeafValues [ -0.24031007751937986, 0.9047619047619048, -0.9245283018867925, -0.8333333333333334 ] SplitGains [ 39.1577375797019, 11.173417128979487, 12.302719747733544 ] NumberOfLeaves 4 NumberOfNodes 3 2 Microsoft.ML.Trainers.FastTree.QuantileRegressionTree LeftChild [ 1, 2, -1 ] RightChild [ -2, -3, -4 ] NumericalSplitFeatureIndexes [ 50, 566, 628 ] NumericalSplitThresholds [ 0.11360833, 0.124133974, 0.052999895 ] CategoricalSplitFlags [ False, False, False ] LeafValues [ -0.7551020408163265, 0.9428571428571428, 0.2, 0.4 ] SplitGains [ 77.13147621155588, 7.7083423229054375, 12.69490786605904 ] NumberOfLeaves 4 NumberOfNodes 3 3 Microsoft.ML.Trainers.FastTree.QuantileRegressionTree LeftChild [ 1, 2, -1 ] RightChild [ -2, -3, -4 ] NumericalSplitFeatureIndexes [ 50, 3873, 103 ] NumericalSplitThresholds [ 0.11360833, 0.1042572, 0.10190575 ] CategoricalSplitFlags [ False, False, False ] LeafValues [ -0.5632183908045977, 0.9259259259259259, 0.2727272727272727, -1 ] SplitGains [ 56.28925136966692, 8.842590132649804, 6.330286019780814 ] NumberOfLeaves 4 NumberOfNodes 3 (values) index type value 0 Microsoft.ML.Transforms.MissingValueReplacingTransformer Microsoft.ML.Transforms.MissingValueReplacingTransformer 1 Microsoft.ML.Transforms.TypeConvertingTransformer Microsoft.ML.Transforms.TypeConvertingTransformer 2 Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer Microsoft.ML.Transforms.Text.TextFeaturizingEstimator+Transformer 3 Microsoft.ML.Data.ColumnConcatenatingTransformer Microsoft.ML.Data.ColumnConcatenatingTransformer 4 Microsoft.ML.Data.BinaryPredictionTransformer<Microsoft.ML.Trainers.FastTree.FastForestBinaryModelParameters> Microsoft.ML.Data.BinaryPredictionTransformer`1[Microsoft.ML.Trainers.FastTree.FastForestBinaryModelParameters] FeatureColumnName Features FeatureColumnType Vector<Single, 5861> Dimensions [ 5861 ] IsKnownSize True ItemType Single RawType System.Single Size 5861 RawType Microsoft.ML.Data.VBuffer<System.Single> Model Microsoft.ML.Trainers.FastTree.FastForestBinaryModelParameters TrainedTreeEnsemble Microsoft.ML.Trainers.FastTree.QuantileRegressionTreeEnsemble Bias 0 TreeWeights [ 1, 1, 1, 1 ] Trees index value 0 Microsoft.ML.Trainers.FastTree.QuantileRegressionTree LeftChild [ 1, 2, -1 ] RightChild [ -2, -3, -4 ] NumericalSplitFeatureIndexes [ 1353, 103, 5857 ] NumericalSplitThresholds [ 0.057353932, 0.10190575, 9.5 ] CategoricalSplitFlags [ False, 

In [13]:
#!transformer-vis model -d 1 -n

In [14]:
var enumTransformer = ((IEnumerable<Microsoft.ML.ITransformer>) model);
var textTransformer = enumTransformer.ToList()[2]; 
#!transformer-vis textTransformer -d 2

## Custom Pipeline

The prior model did fine, but we'd like more control over the text transformation, so we'll provide our own complex pipeline for text processing.

In [15]:
MLContext context = new(seed: seed) {
    GpuDeviceId = 0,
    FallbackToCpu = true,
};
ContextMonitor contextMonitor = context.Monitor();

// Standardize our numeric colums via scaling and imputing missing values
MissingValueReplacingEstimator imputer = context.Transforms.ReplaceMissingValues(columns: colTypes.Numeric.Select(c => new InputOutputColumnPair(c, c)).ToArray(), replacementMode: MissingValueReplacingEstimator.ReplacementMode.DefaultValue);
NormalizingEstimator scaler = context.Transforms.NormalizeRobustScaling(columns: colTypes.Numeric.Select(c => new InputOutputColumnPair(c, c)).ToArray());

// Standardize our boolean columns as singles
TypeConvertingEstimator boolConverter = context.Transforms.Conversion.ConvertType(columns: colTypes.Categorical.Select(c => new InputOutputColumnPair(c, c)).ToArray(), outputKind: DataKind.Single);
    
// Text pre-processing
TextNormalizingEstimator textNormalizer = context.Transforms.Text.NormalizeText(inputColumnName: "Message", outputColumnName: "Message", caseMode: TextNormalizingEstimator.CaseMode.Lower, keepDiacritics: false, keepPunctuations: false, keepNumbers: false);

// Word trigrams / bigrams / unigrams
WordTokenizingEstimator wordTokenizer = context.Transforms.Text.TokenizeIntoWords(inputColumnName: "Message", outputColumnName: "MessageWords");
StopWordsRemovingEstimator stopRemover = context.Transforms.Text.RemoveDefaultStopWords(inputColumnName: "MessageWords", outputColumnName: "MessageWords", language: StopWordsRemovingEstimator.Language.English);
ValueToKeyMappingEstimator labelConverter = context.Transforms.Conversion.MapValueToKey(inputColumnName: "MessageWords", outputColumnName: "MessageWords");
NgramExtractingEstimator ngramExtractor = context.Transforms.Text.ProduceNgrams(inputColumnName: "MessageWords", outputColumnName: "MessageWords", ngramLength: 3, useAllLengths: true, weighting: NgramExtractingEstimator.WeightingCriteria.TfIdf);
LpNormNormalizingEstimator wordNorm = context.Transforms.NormalizeLpNorm(inputColumnName: "MessageWords", outputColumnName: "MessageWords", norm: LpNormNormalizingEstimator.NormFunction.L2);

// Character ngrams
TokenizingByCharactersEstimator charTokenizer = context.Transforms.Text.TokenizeIntoCharactersAsKeys(inputColumnName: "Message", outputColumnName: "MessageChars");
NgramExtractingEstimator charNgram = context.Transforms.Text.ProduceNgrams(inputColumnName: "MessageChars", outputColumnName: "MessageChars", ngramLength: 3, useAllLengths: true, skipLength: 1);
LpNormNormalizingEstimator charNorm = context.Transforms.NormalizeLpNorm(inputColumnName: "MessageChars", outputColumnName: "MessageChars", norm: LpNormNormalizingEstimator.NormFunction.L2);

// We'll concatenate the word and Ngram features together, along with all of our numeric and boolean columns
ColumnConcatenatingEstimator concat = context.Transforms.Concatenate("Features", inputColumnNames: colTypes.Numeric.Concat(colTypes.Categorical).Concat(new[] { "MessageWords", "MessageChars"}).ToArray());

// Since we have scaling in place, let's use all available classifiers
var classifier = context.Auto().BinaryClassification(
    useFastForest: true, 
    useLgbm: true, 
    useFastTree: true, 
    useLbfgsLogisticRegression: true, 
    useSdcaLogisticRegression: true);

// Build a pipeline
SweepablePipeline pipeline = imputer
    .Append(boolConverter)
    .Append(scaler)
    .Append(textNormalizer)
    .Append(wordTokenizer)
    .Append(stopRemover)
    .Append(labelConverter)
    .Append(ngramExtractor)
    .Append(wordNorm)
    .Append(charTokenizer)
    .Append(charNgram)
    .Append(charNorm)
    .Append(concat)
    .Append(classifier);

// Now let's run our experiment using our custom pipeline
var experiment = context.Auto().CreateExperiment()
    .SetPipeline(pipeline)
    .SetDataset(split)
    .SetBinaryClassificationMetric(BinaryClassificationMetric.F1Score, labelColumn: "Label")
    .SetMaxModelToExplore(10);

TrialResult result = await experiment.RunAsync();
ITransformer model = result.Model;

MLCharts.MetricImprovementWithTrials(contextMonitor)

<!-- Plotly chart will be drawn inside this DIV -->

In [16]:
ITransformer model = result.Model;
var scorer = model.Transform(split.TestSet);

// If the model supports calibration, we could use Evaluate instead
var evalResults = context.BinaryClassification.EvaluateNonCalibrated(model.Transform(split.TestSet), labelColumnName: "Label");

// Let's see how it performed
MLCharts.ClassificationReport(evalResults)

<!-- Plotly chart will be drawn inside this DIV -->

In [17]:
model

[ [ Microsoft.ML.Transforms.MissingValueReplacingTransformer, Microsoft.ML.Transforms.TypeConvertingTransformer, Microsoft.ML.Transforms.NormalizingTransformer, Microsoft.ML.Transforms.Text.TextNormalizingTransformer, Microsoft.ML.Transforms.Text.WordTokenizingTransformer, Microsoft.ML.Transforms.Text.StopWordsRemovingTransformer, Microsoft.ML.Transforms.ValueToKeyMappingTransformer, Microsoft.ML.Transforms.Text.NgramExtractingTransformer, Microsoft.ML.Transforms.LpNormNormalizingTransformer, Microsoft.ML.Transforms.Text.TokenizingByCharactersTransformer, Microsoft.ML.Transforms.Text.NgramExtractingTransformer, Microsoft.ML.Transforms.LpNormNormalizingTransformer, Microsoft.ML.Data.ColumnConcatenatingTransformer ], Microsoft.ML.Data.BinaryPredictionTransformer`1[Microsoft.ML.Calibrators.CalibratedModelParametersBase`2[Microsoft.ML.Trainers.FastTree.FastTreeBinaryModelParameters,Microsoft.ML.Calibrators.PlattCalibrator]] ] LastTransformer Microsoft.ML.Data.BinaryPredictionTransformer`1[Microsoft.ML.Calibrators.CalibratedModelParametersBase`2[Microsoft.ML.Trainers.FastTree.FastTreeBinaryModelParameters,Microsoft.ML.Calibrators.PlattCalibrator]] FeatureColumnName Features FeatureColumnType Vector<Single, 9041> Dimensions [ 9041 ] IsKnownSize True ItemType Single RawType System.Single Size 9041 RawType Microsoft.ML.Data.VBuffer<System.Single> Model Microsoft.ML.Calibrators.FeatureWeightsCalibratedModelParameters`2[Microsoft.ML.Trainers.FastTree.FastTreeBinaryModelParameters,Microsoft.ML.Calibrators.PlattCalibrator] SubModel Microsoft.ML.Calibrators.FeatureWeightsCalibratedModelParameters`2[Microsoft.ML.Trainers.FastTree.FastTreeBinaryModelParameters,Microsoft.ML.Calibrators.PlattCalibrator] Calibrator Microsoft.ML.Calibrators.FeatureWeightsCalibratedModelParameters`2[Microsoft.ML.Trainers.FastTree.FastTreeBinaryModelParameters,Microsoft.ML.Calibrators.PlattCalibrator] SubModel Microsoft.ML.Calibrators.FeatureWeightsCalibratedModelParameters`2[Microsoft.ML.Trainers.FastTree.FastTreeBinaryModelParameters,Microsoft.ML.Calibrators.PlattCalibrator] Calibrator Microsoft.ML.Calibrators.FeatureWeightsCalibratedModelParameters`2[Microsoft.ML.Trainers.FastTree.FastTreeBinaryModelParameters,Microsoft.ML.Calibrators.PlattCalibrator] (values) index type value 0 Microsoft.ML.Data.TransformerChain<Microsoft.ML.Data.ColumnConcatenatingTransformer> [ Microsoft.ML.Transforms.MissingValueReplacingTransformer, Microsoft.ML.Transforms.TypeConvertingTransformer, Microsoft.ML.Transforms.NormalizingTransformer, Microsoft.ML.Transforms.Text.TextNormalizingTransformer, Microsoft.ML.Transforms.Text.WordTokenizingTransformer, Microsoft.ML.Transforms.Text.StopWordsRemovingTransformer, Microsoft.ML.Transforms.ValueToKeyMappingTransformer, Microsoft.ML.Transforms.Text.NgramExtractingTransformer, Microsoft.ML.Transforms.LpNormNormalizingTransformer, Microsoft.ML.Transforms.Text.TokenizingByCharactersTransformer, Microsoft.ML.Transforms.Text.NgramExtractingTransformer, Microsoft.ML.Transforms.LpNormNormalizingTransformer, Microsoft.ML.Data.ColumnConcatenatingTransformer ] LastTransformer Microsoft.ML.Data.ColumnConcatenatingTransformer (values) index type value 0 Microsoft.ML.Transforms.MissingValueReplacingTransformer Microsoft.ML.Transforms.MissingValueReplacingTransformer 1 Microsoft.ML.Transforms.TypeConvertingTransformer Microsoft.ML.Transforms.TypeConvertingTransformer 2 Microsoft.ML.Transforms.NormalizingTransformer Microsoft.ML.Transforms.NormalizingTransformer 3 Microsoft.ML.Transforms.Text.TextNormalizingTransformer Microsoft.ML.Transforms.Text.TextNormalizingTransformer 4 Microsoft.ML.Transforms.Text.WordTokenizingTransformer Microsoft.ML.Transforms.Text.WordTokenizingTransformer 5 Microsoft.ML.Transforms.Text.StopWordsRemovingTransformer Microsoft.ML.Transforms.Text.StopWordsRemovingTransformer 6 Microsoft.ML.Transforms.ValueToKeyMappingTransformer Microsoft.ML.Transforms.ValueToKeyMappingTransformer 7 Microsoft.ML.Transforms.Text.NgramExtractingTrans

In [22]:
#!transformer-vis model -d 2 -n

## Deep Learning
This uses TorchSharp and Roberta to fine-tune a model onto our text examples.

In ML.NET this is only available as multi-class classification, so we'll run this as a multi-class classification experiment even though there are only 2 classes.

In [46]:
var dfMultiClass = df.Clone();
dfMultiClass["Label"] = new PrimitiveDataFrameColumn<uint>("Label", dfMultiClass["Label"].Cast<bool>().Select(x => x ? (uint)1 : (uint)0));
dfMultiClass.Sample(5)

index,Label,Message,IsMerge,WorkItems,TotalFiles,ModifiedFiles,AddedFiles,DeletedFiles,TotalLines,NetLines,AddedLines,DeletedLines,HasAddedFiles,HasDeletedFiles,MessageLength,WordCount
0,0,add copyright header,False,0,2,2,0,0,148,6,6,0,False,False,20,3
1,1,Better author testing,False,0,7,5,2,0,356,55,122,67,True,False,21,3
2,0,Mark Microsoft.Extensions.ML as a stable package. (#4356),False,1,2,2,0,0,57,2,2,0,False,False,57,7
3,0,Update dependencies from https://github.com/dotnet/arcade build 20200805.7,False,0,5,5,0,0,1010,36,43,7,False,False,74,6
4,0,Update readme.md,False,0,1,1,0,0,9,0,1,1,False,False,16,2


In [47]:
TorchSharp.torch.InitializeDeviceType(DeviceType.CPU);

MLContext context = new(seed: seed) {
    GpuDeviceId = 0,
    FallbackToCpu = true
};

var split = context.Data.TrainTestSplit(dfMultiClass, testFraction: 0.3, seed: seed);

var valueToKey = context.Transforms.Conversion.MapValueToKey(outputColumnName: "Label", inputColumnName: "Label");
var bert = context.MulticlassClassification.Trainers.TextClassification(sentence1ColumnName: "Message", architecture: BertArchitecture.Roberta, validationSet: split.TestSet);
var keyToValue = context.Transforms.Conversion.MapKeyToValue(outputColumnName: "PredictedLabel", inputColumnName: "PredictedLabel");

// Train our model using this pipeline
var pipeline = valueToKey.Append(bert).Append(keyToValue);
ITransformer model = pipeline.Fit(split.TrainSet);

// Get basics about the model
#!transformer-vis model -n
model

[ Microsoft.ML.Transforms.ValueToKeyMappingTransformer, Microsoft.ML.TorchSharp.NasBert.TextClassificationTransformer, Microsoft.ML.Transforms.KeyToValueMappingTransformer ] LastTransformer Microsoft.ML.Transforms.KeyToValueMappingTransformer (values) index type value 0 Microsoft.ML.Transforms.ValueToKeyMappingTransformer Microsoft.ML.Transforms.ValueToKeyMappingTransformer 1 Microsoft.ML.TorchSharp.NasBert.TextClassificationTransformer Microsoft.ML.TorchSharp.NasBert.TextClassificationTransformer SentenceColumn Microsoft.ML.SchemaShape+Column Name Message Kind Scalar ItemType String RawType System.ReadOnlyMemory<System.Char> IsKey False Annotations [ ] Count 0 (values) (empty) SentenceColumn2 Microsoft.ML.SchemaShape+Column Name <null> Kind Scalar ItemType <null> IsKey False Annotations <null> LabelColumn Microsoft.ML.DataViewSchema+DetachedColumn Name Label Type Key<UInt32, 0-1> Count 2 RawType System.UInt32 Annotations KeyValues Schema [ KeyValues: Vector<UInt32, 2> ] Count 1 (values) index value 0 KeyValues: Vector<UInt32, 2> Name KeyValues Index 0 IsHidden False Type Vector Annotations 2 Microsoft.ML.Transforms.KeyToValueMappingTransformer Microsoft.ML.Transforms.KeyToValueMappingTransformer

In [48]:
var scorer = model.Transform(split.TestSet);
var evalResults = context.MulticlassClassification.Evaluate(model.Transform(split.TestSet));

evalResults

Microsoft.ML.Data.MulticlassClassificationMetrics LogLoss 13.648578322890906 LogLossReduction -20.72279937042476 MacroAccuracy 0.5989684074790458 MicroAccuracy 0.6986301369863014 TopKAccuracy 0 TopKPredictionCount 0 TopKAccuracyForAllK <null> PerClassLogLoss [ 6.748459395132777, 28.182871383487818 ] ConfusionMatrix Microsoft.ML.Data.ConfusionMatrix PerClassPrecision [ 0.7310924369747899, 0.5555555555555556 ] PerClassRecall [ 0.8787878787878788, 0.3191489361702128 ] Counts index value 0 [ 87, 12 ] 1 [ 32, 15 ] NumberOfClasses 2

In [49]:
MLCharts.RenderConfusionMatrix(evalResults.ConfusionMatrix, classNames: new List<string> {"Non-Bugfix", "Bugfix"})

<!-- Plotly chart will be drawn inside this DIV -->

In [50]:
double tp = evalResults.ConfusionMatrix.GetCountForClassPair(1, 1);
double tn = evalResults.ConfusionMatrix.GetCountForClassPair(0, 0);
double fp = evalResults.ConfusionMatrix.GetCountForClassPair(1, 0);
double fn = evalResults.ConfusionMatrix.GetCountForClassPair(0, 1);

double precision = tp / (tp + fp);
double recall = tp / (tp + fn);
double f1Score = 2 * (precision * recall) / (precision + recall);

new {precision, recall, f1Score}

precision,0.5555555555555556
recall,0.3191489361702128
f1Score,0.40540540540540543
